In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
from torch import nn
from torch import optim
import os
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Subset

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")


Training on cpu


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def carregar_dados_no_padding(train_data, test_data, test_size=0.2, random_state=42):
    # Load data from CSV file
    scaler = StandardScaler()

    train_data = pd.read_csv(train_data)
    val_data = pd.read_csv(test_data)

    train_data_filtered = train_data.loc[:, ~(train_data == 0).any(axis=0)]
    val_data_filtered = val_data.loc[:, ~(val_data == 0).any(axis=0)]


    # Split data into features (X) and targets (y) ("hysteresis")
    X = train_data_filtered.drop(['hysteresis', 'joule'], axis=1)
    y = train_data_filtered[['hysteresis', 'joule']]

    # Perform train-test split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=random_state)

    X_test = val_data_filtered.drop(['hysteresis', 'joule'], axis=1)
    y_test = val_data_filtered[['hysteresis', 'joule']]

    X_train = scaler.fit_transform(X_train.values)
    X_val = scaler.transform(X_val.values)
    X_test = scaler.transform(X_test.values)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.BatchNorm1d(input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers, hidden_dim):
        super(Net, self).__init__()
        layers = []

        # Construindo as camadas com base no número de camadas
        for i in range(num_layers):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())

        # Guardar o índice da penúltima camada
        self.penultimate_layer_index = len(layers) - 2

        # Adicionar a camada final de saída
        layers.append(nn.Linear(hidden_dim, output_dim))

        # Definir a sequência de camadas como parte da rede
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        penultimate_output = None

        # Executar o forward manualmente para capturar a penúltima camada
        for i, layer in enumerate(self.network):
            x = layer(x)
            if i == self.penultimate_layer_index:
                penultimate_output = x  # Armazenar a saída da penúltima camada

        # Retornar a saída final e a penúltima camada
        return x, penultimate_output

In [ ]:
from torch import tensor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from torch.optim.lr_scheduler import ReduceLROnPlateau


def train(model,  epochs, optimizer, trainloader, valloader):

    criterion = nn.MSELoss()
    history = {"train_loss": [], "val_loss": []}

    # Coloca o modelo no dispositivo especificado
    model.to(device)

    for epoch in range(epochs):
        # Etapa de treinamento
        model.train()
        train_loss = 0.0

        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs, _ = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)  # Acumula a perda ponderada pelo batch size

        train_loss /= len(trainloader.dataset)  # Média da perda por amostra
        history["train_loss"].append(train_loss)

        # Etapa de validação
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in valloader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs, _ = model(inputs)
                loss = criterion(outputs, targets)

                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(valloader.dataset)  # Média da perda por amostra
        history["val_loss"].append(val_loss)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.8f}, Val Loss: {val_loss:.8f}")

def test(net, testloader):
    """Evaluate the network on the entire test set for a regression problem."""
    mse_criterion = torch.nn.MSELoss()  # Mean Squared Error
    mae_criterion = torch.nn.L1Loss()  # Mean Absolute Error
    total_mse, total_mae, total_mape = 0.0, 0.0, 0.0
    net.eval()

    with torch.no_grad():
        for batch in testloader:
            images, targets = batch[0].to(DEVICE), batch[1].to(DEVICE)
            outputs, _ = net(images)

            # Calculate MSE
            mse = mse_criterion(outputs, targets)
            total_mse += mse.item()

            # Calculate MAE
            mae = mae_criterion(outputs, targets)
            total_mae += mae.item()

            # Calculate MAPE (avoid division by zero)
            epsilon = 1e-8  # Small value to avoid division by zero
            mape = torch.mean(torch.abs((outputs - targets) / (targets + epsilon))) * 100
            total_mape += mape.item()

    # Average metrics per batch
    avg_mse = total_mse / len(testloader)
    avg_mae = total_mae / len(testloader)
    avg_mape = total_mape / len(testloader)

    return avg_mse, avg_mae, avg_mape

In [ ]:
def change_dataset(folder_path):
    folder_path = folder_path
    os.chdir(folder_path)

    X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor = carregar_dados_no_padding('dados_de_treino.csv','dados_de_teste.csv')

    X_train_tensor = X_train_tensor.to(device)
    y_train_tensor = y_train_tensor.to(device)
    X_val_tensor = X_val_tensor.to(device)
    y_val_tensor = y_val_tensor.to(device)
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)

    histeresis_train_tensor = [pair[0].item() for pair in y_train_tensor]
    histeresis_train_tensor = torch.tensor(histeresis_train_tensor, dtype=torch.float32).to(device)
    histeresis_train_tensor = histeresis_train_tensor.unsqueeze(1)

    joule_train_tensor = [pair[1].item() for pair in y_train_tensor]
    joule_train_tensor = torch.tensor(joule_train_tensor, dtype=torch.float32).to(device)
    joule_train_tensor = joule_train_tensor.unsqueeze(1)

    histeresis_val_tensor = [pair[0].item() for pair in y_val_tensor]
    histeresis_val_tensor = torch.tensor(histeresis_val_tensor, dtype=torch.float32).to(device)
    histeresis_val_tensor = histeresis_val_tensor.unsqueeze(1)

    joule_val_tensor = [pair[1].item() for pair in y_val_tensor]
    joule_val_tensor = torch.tensor(joule_val_tensor, dtype=torch.float32).to(device)
    joule_val_tensor = joule_val_tensor.unsqueeze(1)

    histeresis_test_tensor = [pair[0].item() for pair in y_test_tensor]
    histeresis_test_tensor = torch.tensor(histeresis_test_tensor, dtype=torch.float32).to(device)
    histeresis_test_tensor = histeresis_test_tensor.unsqueeze(1)

    joule_test_tensor = [pair[1].item() for pair in y_test_tensor]
    joule_test_tensor = torch.tensor(joule_test_tensor, dtype=torch.float32).to(device)
    joule_test_tensor = joule_test_tensor.unsqueeze(1)

    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor, histeresis_train_tensor, joule_train_tensor, histeresis_val_tensor, joule_val_tensor, histeresis_test_tensor, joule_test_tensor

In [ ]:
X_train_tensorn, y_train_tensorn, X_val_tensorn, y_val_tensorn, X_test_tensorn, y_test_tensorn, histeresis_train_tensorn, joule_train_tensorn, histeresis_val_tensorn, joule_val_tensorn, histeresis_test_tensorn, joule_test_tensorn = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/Nabla')
X_train_tensor2d, y_train_tensor2d, X_val_tensor2d, y_val_tensor2d, X_test_tensor2d, y_test_tensor2d, histeresis_train_tensor2d, joule_train_tensor2d, histeresis_val_tensor2d, joule_val_tensor2d, histeresis_test_tensor2d, joule_test_tensor2d = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/2D')
X_train_tensorv, y_train_tensorv, X_val_tensorv, y_val_tensorv, X_test_tensorv, y_test_tensorv, histeresis_train_tensorv, joule_train_tensorv, histeresis_val_tensorv, joule_val_tensorv, histeresis_test_tensorv, joule_test_tensorv = change_dataset('/content/drive/MyDrive/FederatedLearning-main/Data/IPMSM_datasets/dataset_for_iron_losses_of_IPMSMs/V')


In [ ]:
#datasets para treinamento do autoencoder
NUM_CLIENTS = 10
BATCH_SIZE = 512

auto_train_datasetv = TensorDataset(X_train_tensorv, X_train_tensorv)
auto_val_datasetv = TensorDataset(X_val_tensorv, X_val_tensorv)
auto_test_datasetv = TensorDataset(X_test_tensorv, X_test_tensorv)

auto_train_datasetn = TensorDataset(X_train_tensorn, X_train_tensorn)
auto_val_datasetn = TensorDataset(X_val_tensorn, X_val_tensorn)
auto_test_datasetn = TensorDataset(X_test_tensorn, X_test_tensorn)

auto_train_dataset2d = TensorDataset(X_train_tensor2d, X_train_tensor2d)
auto_val_dataset2d = TensorDataset(X_val_tensor2d,X_val_tensor2d)
auto_test_dataset2d = TensorDataset(X_test_tensor2d, X_test_tensor2d)

In [ ]:
batch_size = 256
auto_train_loader_v = DataLoader(auto_train_datasetv, batch_size=batch_size, shuffle=True)
auto_val_loader_v = DataLoader(auto_val_datasetv, batch_size=batch_size, shuffle=False)
auto_test_loader_v = DataLoader(auto_test_datasetv, batch_size=batch_size, shuffle=False)

auto_train_loader_n = DataLoader(auto_train_datasetn, batch_size=batch_size, shuffle=True)
auto_val_loader_n = DataLoader(auto_val_datasetn, batch_size=batch_size, shuffle=False)
auto_test_loader_n = DataLoader(auto_test_datasetn, batch_size=batch_size, shuffle=False)

auto_train_loader_2d = DataLoader(auto_train_dataset2d, batch_size=batch_size, shuffle=True)
auto_val_loader_2d = DataLoader(auto_val_dataset2d, batch_size=batch_size, shuffle=False)
auto_test_loader_2d = DataLoader(auto_test_dataset2d, batch_size=batch_size, shuffle=False)

In [ ]:
modeln = Autoencoder(X_train_tensorn.shape[1],20).to(device)
optimizern = optim.Adam(modeln.parameters(), lr=0.001)

model2d = Autoencoder(X_train_tensor2d.shape[1],20).to(device)
optimizer2d = optim.Adam(model2d.parameters(), lr=0.001)

modelv = Autoencoder(X_train_tensorv.shape[1],20).to(device)
optimizerv = optim.Adam(modelv.parameters(), lr=0.001)

In [ ]:
train(modeln,100,optimizern,auto_train_loader_n,auto_val_loader_n)

Epoch 1/100, Train Loss: 0.49396852, Val Loss: 0.14536724
Epoch 2/100, Train Loss: 0.08797004, Val Loss: 0.05586946
Epoch 3/100, Train Loss: 0.04469657, Val Loss: 0.03752718
Epoch 4/100, Train Loss: 0.03057485, Val Loss: 0.02560651
Epoch 5/100, Train Loss: 0.02359644, Val Loss: 0.01959598
Epoch 6/100, Train Loss: 0.01918421, Val Loss: 0.02157160
Epoch 7/100, Train Loss: 0.01656270, Val Loss: 0.01542758
Epoch 8/100, Train Loss: 0.01548370, Val Loss: 0.01484034
Epoch 9/100, Train Loss: 0.01341693, Val Loss: 0.01341088
Epoch 10/100, Train Loss: 0.01257294, Val Loss: 0.00993941
Epoch 11/100, Train Loss: 0.01143115, Val Loss: 0.01312815
Epoch 12/100, Train Loss: 0.01153370, Val Loss: 0.00947460
Epoch 13/100, Train Loss: 0.01077107, Val Loss: 0.01163956
Epoch 14/100, Train Loss: 0.01002872, Val Loss: 0.00801798
Epoch 15/100, Train Loss: 0.00937119, Val Loss: 0.01844568
Epoch 16/100, Train Loss: 0.00890498, Val Loss: 0.00624872
Epoch 17/100, Train Loss: 0.00828811, Val Loss: 0.00537585
Epoch 

In [ ]:
train(model2d,100,optimizer2d,auto_train_loader_2d,auto_val_loader_2d)

Epoch 1/100, Train Loss: 0.48135274, Val Loss: 0.16304847
Epoch 2/100, Train Loss: 0.10140368, Val Loss: 0.06322409
Epoch 3/100, Train Loss: 0.04826714, Val Loss: 0.03339975
Epoch 4/100, Train Loss: 0.03088795, Val Loss: 0.02287472
Epoch 5/100, Train Loss: 0.02411219, Val Loss: 0.01795786
Epoch 6/100, Train Loss: 0.01935990, Val Loss: 0.01373549
Epoch 7/100, Train Loss: 0.01631243, Val Loss: 0.01095359
Epoch 8/100, Train Loss: 0.01388065, Val Loss: 0.00906565
Epoch 9/100, Train Loss: 0.01199449, Val Loss: 0.00696045
Epoch 10/100, Train Loss: 0.01055200, Val Loss: 0.00607778
Epoch 11/100, Train Loss: 0.00939345, Val Loss: 0.00440871
Epoch 12/100, Train Loss: 0.00839422, Val Loss: 0.00442155
Epoch 13/100, Train Loss: 0.00785306, Val Loss: 0.00282711
Epoch 14/100, Train Loss: 0.00748597, Val Loss: 0.00326777
Epoch 15/100, Train Loss: 0.00685892, Val Loss: 0.00279802
Epoch 16/100, Train Loss: 0.00643251, Val Loss: 0.00264245
Epoch 17/100, Train Loss: 0.00631969, Val Loss: 0.00156591
Epoch 

In [ ]:
train(modelv,100,optimizerv,auto_train_loader_v,auto_val_loader_v)

Epoch 1/100, Train Loss: 0.42922436, Val Loss: 0.05932092
Epoch 2/100, Train Loss: 0.03890598, Val Loss: 0.02430960
Epoch 3/100, Train Loss: 0.02407187, Val Loss: 0.01697816
Epoch 4/100, Train Loss: 0.01846704, Val Loss: 0.01246126
Epoch 5/100, Train Loss: 0.01573166, Val Loss: 0.01092683
Epoch 6/100, Train Loss: 0.01390230, Val Loss: 0.00899129
Epoch 7/100, Train Loss: 0.01256769, Val Loss: 0.00757972
Epoch 8/100, Train Loss: 0.01182020, Val Loss: 0.00690855
Epoch 9/100, Train Loss: 0.01146150, Val Loss: 0.00605038
Epoch 10/100, Train Loss: 0.01039598, Val Loss: 0.00595120
Epoch 11/100, Train Loss: 0.00995472, Val Loss: 0.00502361
Epoch 12/100, Train Loss: 0.00942263, Val Loss: 0.00465700
Epoch 13/100, Train Loss: 0.00922712, Val Loss: 0.00423271
Epoch 14/100, Train Loss: 0.00904476, Val Loss: 0.00385799
Epoch 15/100, Train Loss: 0.00880287, Val Loss: 0.00336743
Epoch 16/100, Train Loss: 0.00772337, Val Loss: 0.00332110
Epoch 17/100, Train Loss: 0.00753353, Val Loss: 0.00293731
Epoch 

In [ ]:
test(modeln,auto_test_loader_n)

(0.00984552399696488, 0.07709810765166032, 140.6575545260781)

In [ ]:
test(model2d,auto_test_loader_2d)

(0.000989463768507305, 0.023799514692080647, 28.959233283996582)

In [ ]:
test(modelv,auto_test_loader_v)

(0.0003878631840435494, 0.01717522142356948, 10.143960777081942)

In [ ]:
modeln.eval()
modelv.eval()
model2d.eval()

encoded_dataset_n = []
encoded_dataset_v = []
encoded_dataset_2d = []

vencoded_dataset_n = []
vencoded_dataset_v = []
vencoded_dataset_2d = []

tencoded_dataset_n = []
tencoded_dataset_v = []
tencoded_dataset_2d = []

with torch.no_grad():
    for x in X_train_tensorn:
        x = x.unsqueeze(0).to(device)
        latent = modeln.encoder(x)
        encoded_dataset_n.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_train_tensor2d:
        x = x.unsqueeze(0).to(device)
        latent = model2d.encoder(x)
        encoded_dataset_2d.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_train_tensorv:
        x = x.unsqueeze(0).to(device)
        latent = modelv.encoder(x)
        encoded_dataset_v.append(latent.squeeze(0).cpu())

encoded_dataset_n = torch.stack(encoded_dataset_n)
encoded_dataset_2d = torch.stack(encoded_dataset_2d)
encoded_dataset_v = torch.stack(encoded_dataset_v)

print(f"Shape do dataset codificado: {encoded_dataset_n.shape}")
print(f"Shape do dataset codificado: {encoded_dataset_2d.shape}")
print(f"Shape do dataset codificado: {encoded_dataset_v.shape}")

with torch.no_grad():
    for x in X_val_tensorn:
        x = x.unsqueeze(0).to(device)
        latent = modeln.encoder(x)
        vencoded_dataset_n.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_val_tensor2d:
        x = x.unsqueeze(0).to(device)
        latent = model2d.encoder(x)
        vencoded_dataset_2d.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_val_tensorv:
        x = x.unsqueeze(0).to(device)
        latent = modelv.encoder(x)
        vencoded_dataset_v.append(latent.squeeze(0).cpu())

vencoded_dataset_n = torch.stack(vencoded_dataset_n)
vencoded_dataset_2d = torch.stack(vencoded_dataset_2d)
vencoded_dataset_v = torch.stack(vencoded_dataset_v)

print(f"Shape do dataset codificado: {vencoded_dataset_n.shape}")
print(f"Shape do dataset codificado: {vencoded_dataset_2d.shape}")
print(f"Shape do dataset codificado: {vencoded_dataset_v.shape}")

with torch.no_grad():
    for x in X_test_tensorn:
        x = x.unsqueeze(0).to(device)
        latent = modeln.encoder(x)
        tencoded_dataset_n.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_test_tensor2d:
        x = x.unsqueeze(0).to(device)
        latent = model2d.encoder(x)
        tencoded_dataset_2d.append(latent.squeeze(0).cpu())

with torch.no_grad():
    for x in X_test_tensorv:
        x = x.unsqueeze(0).to(device)
        latent = modelv.encoder(x)
        tencoded_dataset_v.append(latent.squeeze(0).cpu())

tencoded_dataset_n = torch.stack(tencoded_dataset_n)
tencoded_dataset_2d = torch.stack(tencoded_dataset_2d)
tencoded_dataset_v = torch.stack(tencoded_dataset_v)

print(f"Shape do dataset codificado: {tencoded_dataset_n.shape}")
print(f"Shape do dataset codificado: {tencoded_dataset_2d.shape}")
print(f"Shape do dataset codificado: {tencoded_dataset_v.shape}")


Shape do dataset codificado: torch.Size([18436, 20])
Shape do dataset codificado: torch.Size([19328, 20])
Shape do dataset codificado: torch.Size([19026, 20])
Shape do dataset codificado: torch.Size([4610, 20])
Shape do dataset codificado: torch.Size([4832, 20])
Shape do dataset codificado: torch.Size([4757, 20])
Shape do dataset codificado: torch.Size([4609, 20])
Shape do dataset codificado: torch.Size([4831, 20])
Shape do dataset codificado: torch.Size([4756, 20])


In [ ]:
from torch.utils.data import ConcatDataset

train_datasetv = TensorDataset(encoded_dataset_v, y_train_tensorv)
val_datasetv = TensorDataset(vencoded_dataset_v, y_val_tensorv)
test_datasetv = TensorDataset(tencoded_dataset_v, y_test_tensorv)

train_datasetn = TensorDataset(encoded_dataset_n, y_train_tensorn)
val_datasetn = TensorDataset(vencoded_dataset_n, y_val_tensorn)
test_datasetn = TensorDataset(tencoded_dataset_n, y_test_tensorn)

train_dataset2d = TensorDataset(encoded_dataset_2d, y_train_tensor2d)
val_dataset2d = TensorDataset(vencoded_dataset_2d, y_val_tensor2d)
test_dataset2d = TensorDataset(tencoded_dataset_2d, y_test_tensor2d)

all_train = ConcatDataset([train_datasetv,train_datasetn,train_dataset2d])
all_val = ConcatDataset([val_datasetv,val_datasetn,val_dataset2d])

In [ ]:
train_loader = DataLoader(all_train, batch_size=128, shuffle=True)
val_loader = DataLoader(all_val, batch_size=128, shuffle=True)

In [ ]:
# Mostrando formato do dataset de treinamento
train_sample, train_label = next(iter(train_loader))
print(f"Formato das amostras de treinamento: {train_sample.shape}")
print(f"Formato dos rótulos de treinamento: {train_label.shape}")

# Mostrando formato do dataset de validação
val_sample, val_label = next(iter(val_loader))
print(f"Formato das amostras de validação: {val_sample.shape}")
print(f"Formato dos rótulos de validação: {val_label.shape}")

Formato das amostras de treinamento: torch.Size([128, 20])
Formato dos rótulos de treinamento: torch.Size([128, 2])
Formato das amostras de validação: torch.Size([128, 20])
Formato dos rótulos de validação: torch.Size([128, 2])


In [ ]:
model = Net(20,2,10,30).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

Net(
  (network): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=30, bias=True)
    (5): ReLU()
    (6): Linear(in_features=30, out_features=30, bias=True)
    (7): ReLU()
    (8): Linear(in_features=30, out_features=30, bias=True)
    (9): ReLU()
    (10): Linear(in_features=30, out_features=30, bias=True)
    (11): ReLU()
    (12): Linear(in_features=30, out_features=30, bias=True)
    (13): ReLU()
    (14): Linear(in_features=30, out_features=30, bias=True)
    (15): ReLU()
    (16): Linear(in_features=30, out_features=30, bias=True)
    (17): ReLU()
    (18): Linear(in_features=30, out_features=30, bias=True)
    (19): ReLU()
    (20): Linear(in_features=30, out_features=2, bias=True)
  )
)


In [ ]:
train(model,1000,optimizer,train_loader,val_loader)


Epoch 1/1000, Train Loss: 0.37312496, Val Loss: 0.11285029
Epoch 2/1000, Train Loss: 0.10266461, Val Loss: 0.09197535
Epoch 3/1000, Train Loss: 0.08933942, Val Loss: 0.08491936
Epoch 4/1000, Train Loss: 0.08064202, Val Loss: 0.07435031
Epoch 5/1000, Train Loss: 0.07491142, Val Loss: 0.06953446
Epoch 6/1000, Train Loss: 0.07011239, Val Loss: 0.06662471
Epoch 7/1000, Train Loss: 0.06502358, Val Loss: 0.06074925
Epoch 8/1000, Train Loss: 0.05837612, Val Loss: 0.05175272
Epoch 9/1000, Train Loss: 0.03550159, Val Loss: 0.02647797
Epoch 10/1000, Train Loss: 0.02512510, Val Loss: 0.02224708
Epoch 11/1000, Train Loss: 0.02197079, Val Loss: 0.02073403
Epoch 12/1000, Train Loss: 0.02030432, Val Loss: 0.02131954
Epoch 13/1000, Train Loss: 0.01898344, Val Loss: 0.01768329
Epoch 14/1000, Train Loss: 0.01739534, Val Loss: 0.01746300
Epoch 15/1000, Train Loss: 0.01665304, Val Loss: 0.01511445
Epoch 16/1000, Train Loss: 0.01617434, Val Loss: 0.01621137
Epoch 17/1000, Train Loss: 0.01550439, Val Loss: 

In [ ]:
test(model, test_datasetv)

(0.0015393306145391758, 0.026174361996078133, 21.160047367865282)

In [ ]:
test(model, test_datasetn)

(0.004598289515796319, 0.03975924910463427, 61.700655137391)

In [ ]:
test(model, test_dataset2d)

(0.10414411214415521, 0.0290539539158138, 14.06480612977568)